In [8]:
import pandas as pd
from transformers import pipeline
import re
import kss

/Users/apple/Softeer-DE/W2/w2m5_team/lib/python3.14/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:
import emoji

In [42]:
import numpy as np

In [9]:
df = pd.read_csv("reviews.csv")
df.head()

,지점명,닉네임,작성리뷰수,작성시간,방문일,방문횟수,인증수단,좋아요수,리뷰내용
0,신촌2호점,ok_****,466,25.11.27.목,점심에 방문,1번째 방문,영수증,0,멀리서 온 친구랑 술한잔 하고싶어서 방문했는데\n음식맛은 기본 한다는 더본 홍콩반점...
1,신촌2호점,rmdlll,31,25.12.2.화,저녁에 방문,1번째 방문,영수증,0,"저렴한 맛에 가는 곳 •••\n특별히 너무 맛있지는 않지만, 어떤 메뉴를 시켜도 맛..."
2,신촌2호점,재재슈슈,3,25.11.27.목,저녁에 방문,3번째 방문,영수증,0,신촌에서 친구랑 가볍게 한잔 하고싶어서 갈곳을 찾다가 딱걸린 홍콩반점.깐풍새우 소스...
3,신촌2호점,엉클뤼치,8,25.11.28.금,점심에 방문,4번째 방문,영수증,0,볶음밥 짱 맛있어요. 강추합니다\n직원분들도 친절하고 매장도 깨끗.\n재방문 예정 ...
4,신촌2호점,엉클뤼치,8,25.10.1.수,저녁에 방문,3번째 방문,영수증,0,정말 맛있게 잘 먹었습니다. 식당가격들이 다 올라서 힘든데 홍콩반점은 착한가격에 양...


In [10]:
df.describe()

,지점명,닉네임,작성리뷰수,작성시간,방문일,방문횟수,인증수단,좋아요수,리뷰내용
count,3214,3214,3214,3214,2983,2983,2983,3214,3036
unique,13,2237,1292,1413,5,131,3,11,1930
top,신촌2호점,hee302137,"1,198",23.4.14.금,점심에 방문,1번째 방문,영수증,0,굿
freq,310,65,66,21,1819,1884,2718,3080,272


In [11]:
df_review = df[['지점명', '리뷰내용']]
df_review.head(10)

,지점명,리뷰내용
0,신촌2호점,멀리서 온 친구랑 술한잔 하고싶어서 방문했는데\n음식맛은 기본 한다는 더본 홍콩반점...
1,신촌2호점,"저렴한 맛에 가는 곳 •••\n특별히 너무 맛있지는 않지만, 어떤 메뉴를 시켜도 맛..."
2,신촌2호점,신촌에서 친구랑 가볍게 한잔 하고싶어서 갈곳을 찾다가 딱걸린 홍콩반점.깐풍새우 소스...
3,신촌2호점,볶음밥 짱 맛있어요. 강추합니다\n직원분들도 친절하고 매장도 깨끗.\n재방문 예정 ...
4,신촌2호점,정말 맛있게 잘 먹었습니다. 식당가격들이 다 올라서 힘든데 홍콩반점은 착한가격에 양...
5,신촌2호점,탕짜면 맛있어요🖤
6,신촌2호점,날이 선선해지니 뜨거운 짬뽕국물이 그리워져서 혼밥하기 좋은 홍콩반점 다녀왔어요. 속...
7,신촌2호점,탕수육 정말 맛있어요~
8,신촌2호점,탕슈욱은 홍콩반점이 제일 맛있어요.~~ 튀김옷도 적당하고 색깔도 연하게 노릿노릿 한...
9,신촌2호점,음식 나오는 카운터에서 손톱 깎는 건 정말 아닌 것 같습니다. 위생적으로 굉장히 불...


In [18]:
review = df_review.loc[0, '리뷰내용']
review

'멀리서 온 친구랑 술한잔 하고싶어서 방문했는데\n음식맛은 기본 한다는 더본 홍콩반점\n가맹점 직원 교육은 없나요\n너무 불친절해서 음식 퀄리티마저 최악으로 떨어지네요😡😠😡😠😡😠\n이렇게 불만족 했던적은 처음이네요\n기분나쁘게 제돈쓰며 먹고갑니다\n다시는 안오고 싶은 곳이네요'

In [38]:
def split_review_sentences(review):
    if not isinstance(review, str):
        return []
    
    lines = [l.strip() for l in review.split('\n') if l.strip()]
    sentences = []
    for line in lines:
        sentences.extend(kss.split_sentences(line))
    return sentences

In [20]:
sentences = split_review_sentences(review)
sentences

['멀리서 온 친구랑 술한잔 하고싶어서 방문했는데',
 '음식맛은 기본 한다는 더본 홍콩반점',
 '가맹점 직원 교육은 없나요',
 '너무 불친절해서 음식 퀄리티마저 최악으로 떨어지네요😡😠😡😠😡😠',
 '이렇게 불만족 했던적은 처음이네요',
 '기분나쁘게 제돈쓰며 먹고갑니다',
 '다시는 안오고 싶은 곳이네요']

In [39]:
def preprocess_review(review):
    sentences = split_review_sentences(review)
    sentences = [emoji.demojize(s) for s in sentences]
    return sentences

In [25]:
sentence = preprocess_review(review)
sentence

['멀리서 온 친구랑 술한잔 하고싶어서 방문했는데',
 '음식맛은 기본 한다는 더본 홍콩반점',
 '가맹점 직원 교육은 없나요',
 '너무 불친절해서 음식 퀄리티마저 최악으로 떨어지네요:pouting_face::angry_face::pouting_face::angry_face::pouting_face::angry_face:',
 '이렇게 불만족 했던적은 처음이네요',
 '기분나쁘게 제돈쓰며 먹고갑니다',
 '다시는 안오고 싶은 곳이네요']

In [27]:
sentiment_model = pipeline(
    "sentiment-analysis",
    model="nlptown/bert-base-multilingual-uncased-sentiment"
)

results = sentiment_model(sentence)
results

Device set to use mps:0


[{'label': '5 stars', 'score': 0.2714393436908722},
 {'label': '4 stars', 'score': 0.3202765882015228},
 {'label': '1 star', 'score': 0.4890397787094116},
 {'label': '2 stars', 'score': 0.4676998257637024},
 {'label': '1 star', 'score': 0.6370121240615845},
 {'label': '5 stars', 'score': 0.3201994001865387},
 {'label': '5 stars', 'score': 0.43130549788475037}]

In [50]:
review_conf = 0
review_sum = 0
for element in results:
    if element['score'] >= 0.4:
        print(f"star: {element['label']} score: {element['score']}")
        review_conf += element['score']
        review_sum += int(element['label'][0]) * element['score']

print(review_sum)
print(review_conf)
print(review_sum / review_conf)

star: 1 star score: 0.4890397787094116
star: 2 stars score: 0.4676998257637024
star: 1 star score: 0.6370121240615845
star: 5 stars score: 0.43130549788475037
4.217979043722153
2.025057226419449
2.0828937516892103


In [53]:
def review_star_from_text(review, threshold=0.4):
    sentences = preprocess_review(review)

    if len(sentences) == 0:
        return np.nan

    results = sentiment_model(sentences)

    review_sum = 0
    review_conf = 0

    for element in results:
        if element['score'] >= threshold:
            star = int(element['label'][0])
            review_sum += star * element['score']
            review_conf += element['score']

    # fallback: 전부 confidence 낮을 때
    if review_conf == 0:
        for element in results:
            star = int(element['label'][0])
            review_sum += star
        return review_sum / len(results)

    return review_sum / review_conf

In [54]:
df_review['predicted_star'] = df_review['리뷰내용'].apply(review_star_from_text)

/var/folders/gy/mwkqsrr91251dr8cyvh7jhy00000gn/T/ipykernel_21010/3305882929.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_review['predicted_star'] = df_review['리뷰내용'].apply(review_star_from_text)


In [59]:
def star_to_label(star):
    if star < 3:
        return 'Negative'
    elif star < 3.5:
        return 'neutral'
    else:
        return 'Positive'

In [60]:
df_review['sentiment'] = df_review['predicted_star'].apply(star_to_label)

/var/folders/gy/mwkqsrr91251dr8cyvh7jhy00000gn/T/ipykernel_21010/1932407530.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_review['sentiment'] = df_review['predicted_star'].apply(star_to_label)


In [61]:
df_review.head(30)

,지점명,리뷰내용,predicted_star,sentiment
0,신촌2호점,멀리서 온 친구랑 술한잔 하고싶어서 방문했는데\n음식맛은 기본 한다는 더본 홍콩반점...,2.082894,Negative
1,신촌2호점,"저렴한 맛에 가는 곳 •••\n특별히 너무 맛있지는 않지만, 어떤 메뉴를 시켜도 맛...",3.000000,neutral
2,신촌2호점,신촌에서 친구랑 가볍게 한잔 하고싶어서 갈곳을 찾다가 딱걸린 홍콩반점.깐풍새우 소스...,1.502503,Negative
3,신촌2호점,볶음밥 짱 맛있어요. 강추합니다\n직원분들도 친절하고 매장도 깨끗.\n재방문 예정 ...,5.000000,Positive
4,신촌2호점,정말 맛있게 잘 먹었습니다. 식당가격들이 다 올라서 힘든데 홍콩반점은 착한가격에 양...,2.829253,Negative
5,신촌2호점,탕짜면 맛있어요🖤,1.000000,Negative
6,신촌2호점,날이 선선해지니 뜨거운 짬뽕국물이 그리워져서 혼밥하기 좋은 홍콩반점 다녀왔어요. 속...,5.000000,Positive
7,신촌2호점,탕수육 정말 맛있어요~,3.000000,neutral
8,신촌2호점,탕슈욱은 홍콩반점이 제일 맛있어요.~~ 튀김옷도 적당하고 색깔도 연하게 노릿노릿 한...,2.000000,Negative
9,신촌2호점,음식 나오는 카운터에서 손톱 깎는 건 정말 아닌 것 같습니다. 위생적으로 굉장히 불...,1.000000,Negative


In [46]:
df_review.head(10)

,지점명,리뷰내용,predicted_star,sentiment
0,신촌2호점,멀리서 온 친구랑 술한잔 하고싶어서 방문했는데\n음식맛은 기본 한다는 더본 홍콩반점...,1.293458,negative
1,신촌2호점,"저렴한 맛에 가는 곳 •••\n특별히 너무 맛있지는 않지만, 어떤 메뉴를 시켜도 맛...",3.000000,neutral
2,신촌2호점,신촌에서 친구랑 가볍게 한잔 하고싶어서 갈곳을 찾다가 딱걸린 홍콩반점.깐풍새우 소스...,2.666667,negative
3,신촌2호점,볶음밥 짱 맛있어요. 강추합니다\n직원분들도 친절하고 매장도 깨끗.\n재방문 예정 ...,5.000000,positive
4,신촌2호점,정말 맛있게 잘 먹었습니다. 식당가격들이 다 올라서 힘든데 홍콩반점은 착한가격에 양...,1.953139,negative
5,신촌2호점,탕짜면 맛있어요🖤,1.000000,negative
6,신촌2호점,날이 선선해지니 뜨거운 짬뽕국물이 그리워져서 혼밥하기 좋은 홍콩반점 다녀왔어요. 속...,5.000000,positive
7,신촌2호점,탕수육 정말 맛있어요~,3.000000,neutral
8,신촌2호점,탕슈욱은 홍콩반점이 제일 맛있어요.~~ 튀김옷도 적당하고 색깔도 연하게 노릿노릿 한...,2.000000,negative
9,신촌2호점,음식 나오는 카운터에서 손톱 깎는 건 정말 아닌 것 같습니다. 위생적으로 굉장히 불...,1.000000,negative


In [49]:
review = df_review.loc[4, '리뷰내용']
review

'정말 맛있게 잘 먹었습니다. 식당가격들이 다 올라서 힘든데 홍콩반점은 착한가격에 양도 푸짐하고 너무 좋아요. 대만족. 자주 올게요'

In [62]:
POSITIVE_KEYWORDS = [
    '맛있', '좋', '만족', '추천', '친절',
    '괜찮', '훌륭', '최고', '강추'
]

NEGATIVE_KEYWORDS = [
    '최악', '불만족', '불친절', '별로',
    '다시는', '안오', '기분나쁘',
    '형편없', '실망', '위생'
]

NEGATIVE_EMOJI = [
    ':angry_face:', ':pouting_face:', ':face_with_symbols_on_mouth:'
]

In [63]:
def keyword_adjustment(sentence):
    score = 0

    for w in POSITIVE_KEYWORDS:
        if w in sentence:
            score += 1.0   # 긍정은 +1

    for w in NEGATIVE_KEYWORDS:
        if w in sentence:
            score -= 1.5   # 부정은 더 강하게

    for e in NEGATIVE_EMOJI:
        score -= sentence.count(e) * 0.5

    return score

def adjusted_star(sentence, model_output):
    base_star = int(model_output['label'][0])
    conf = model_output['score']

    adj = keyword_adjustment(sentence)

    # star 보정
    final_star = base_star + adj

    # 별점 범위 제한
    final_star = max(1, min(5, final_star))

    return final_star, conf

In [64]:
def review_star_from_text(review, threshold=0.45):
    sentences = preprocess_review(review)

    if len(sentences) == 0:
        return np.nan

    results = sentiment_model(sentences)

    review_sum = 0
    review_conf = 0

    for sentence, result in zip(sentences, results):
        if result['score'] >= threshold:
            star, conf = adjusted_star(sentence, result)
            review_sum += star * conf
            review_conf += conf

    if review_conf == 0:
        stars = [adjusted_star(s, r)[0] for s, r in zip(sentences, results)]
        return np.mean(stars)

    return review_sum / review_conf


In [65]:
df_review['predicted_star'] = df_review['리뷰내용'].apply(review_star_from_text)

/var/folders/gy/mwkqsrr91251dr8cyvh7jhy00000gn/T/ipykernel_21010/3305882929.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_review['predicted_star'] = df_review['리뷰내용'].apply(review_star_from_text)


In [66]:
def star_to_label(star):
    if star < 3:
        return 'Negative'
    elif star < 3.5:
        return 'neutral'
    else:
        return 'Positive'

In [67]:
df_review['sentiment'] = df_review['predicted_star'].apply(star_to_label)

/var/folders/gy/mwkqsrr91251dr8cyvh7jhy00000gn/T/ipykernel_21010/1932407530.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_review['sentiment'] = df_review['predicted_star'].apply(star_to_label)


In [68]:
df_review.head(30)

,지점명,리뷰내용,predicted_star,sentiment
0,신촌2호점,멀리서 온 친구랑 술한잔 하고싶어서 방문했는데\n음식맛은 기본 한다는 더본 홍콩반점...,1.000000,Negative
1,신촌2호점,"저렴한 맛에 가는 곳 •••\n특별히 너무 맛있지는 않지만, 어떤 메뉴를 시켜도 맛...",4.000000,Positive
2,신촌2호점,신촌에서 친구랑 가볍게 한잔 하고싶어서 갈곳을 찾다가 딱걸린 홍콩반점.깐풍새우 소스...,3.000000,neutral
3,신촌2호점,볶음밥 짱 맛있어요. 강추합니다\n직원분들도 친절하고 매장도 깨끗.\n재방문 예정 ...,5.000000,Positive
4,신촌2호점,정말 맛있게 잘 먹었습니다. 식당가격들이 다 올라서 힘든데 홍콩반점은 착한가격에 양...,2.953139,Negative
5,신촌2호점,탕짜면 맛있어요🖤,2.000000,Negative
6,신촌2호점,날이 선선해지니 뜨거운 짬뽕국물이 그리워져서 혼밥하기 좋은 홍콩반점 다녀왔어요. 속...,5.000000,Positive
7,신촌2호점,탕수육 정말 맛있어요~,4.000000,Positive
8,신촌2호점,탕슈욱은 홍콩반점이 제일 맛있어요.~~ 튀김옷도 적당하고 색깔도 연하게 노릿노릿 한...,2.666667,Negative
9,신촌2호점,음식 나오는 카운터에서 손톱 깎는 건 정말 아닌 것 같습니다. 위생적으로 굉장히 불...,1.000000,Negative


In [73]:
review = df_review.loc[25, '리뷰내용']
review

'짜장면이 너무 맛있어서 자주 방문하는 곳이에요! 다들 고기짜장에 고춧가루 뿌려드세요!!! 사장님도 너무 친절하셔서 좋아요~~☺️'

In [75]:
from konlpy.tag import Mecab
mecab = Mecab()

Exception: Install MeCab in order to use it: http://konlpy.org/en/latest/install/